# Option 1: Foreign catalog

### Create a connection
A connection specifies a path and credentials for accessing an external database system. To create a connection, you can use Catalog Explorer or the CREATE CONNECTION SQL command in a Databricks notebook or the Databricks SQL query editor.

https://docs.databricks.com/aws/en/query-federation/postgresql?language=SQL

In [0]:
%sql
-- Step 1: Create a connection to the PostgreSQL database using JDBC
CREATE CONNECTION IF NOT EXISTS conn
TYPE POSTGRESQL
OPTIONS (
  host = 'host',
  port = 'port',
  user = 'user',
  password = 'password'
);

SHOW CONNECTIONS;

### Create a foreign catalog
A foreign catalog mirrors a database in an external data system so that you can query and manage access to data in that database using Databricks and Unity Catalog. To create a foreign catalog, you use a connection to the data source that has already been defined.

Permissions required: CREATE CATALOG permission on the metastore and either ownership of the connection or the CREATE FOREIGN CATALOG privilege on the connection.

In [0]:
%sql
CREATE FOREIGN CATALOG IF NOT EXISTS foreign_catalog USING CONNECTION conn
OPTIONS (database 'db');

# Option 2: Only table connection

### Create a catalog
Create a standard Unity Catalog catalog. If the catalog already contains Delta tables, you cannot use the USING JDBC approach to register external tables in the same schema as those Delta tables. Unity Catalog does not allow mixing managed (Delta/Iceberg) tables and external JDBC tables in the same schema or catalog. Attempting to do so will result in an error.

Permissions required: CREATE CATALOG permission on the metastore.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS catalog
COMMENT 'Catalog for external postgresql tables';

Here in a csv, we have the list of tables to register. The csv has been uploaded in our volume.

In [0]:
# read the information using a spark dataframe
df = (spark.read
      .option("header","true")
      .option("delimiter",";")
      .csv('/Volumes/tables.csv')
)
df.limit(5).display()

In [0]:
# create a list of the name of schemas and a list of the name of tables, both from the spark dataframe
schemas = [row['nom_schema'] for row in df.select('nom_schema').collect()]
tables = [row['nom_table'] for row in df.select('nom_table').collect()]

The following approach registers the table in the Databricks catalog and queries will be pushed down to PostgreSQL, so data is not ingested into Databricks storage:

(This requires Unity Catalog and the JDBC table feature enabled in your workspace.)

In [0]:
display(spark.sql("USE CATALOG catalog"))

# register the table in the Databricks catalog
for schema, table in zip(schemas, tables):
    spqrk.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
    table = f"{schema}.{table}"
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS catalog.{table}
    USING JDBC
    OPTIONS (
        url 'jdbc:postgresql://host:port/db',
        dbtable '{table}'
        user 'user',
        password 'password',
        driver 'org.postgresql.Driver'
    )
""")

The JDBC table feature in Databricks Unity Catalog allows you to register external tables that reference data in JDBC-accessible databases (like PostgreSQL) without ingesting the data into Databricks storage. This feature must be enabled in your workspace to use SQL statements like `CREATE TABLE ... USING JDBC ...` that register external tables in Unity Catalog and push queries down to the source database.

To check or enable this feature, you need to:

- Be on a supported Databricks Runtime.
- Have Unity Catalog enabled in your workspace.
- Have the JDBC table feature enabled by your Databricks admin.

This is a workspace-level setting that may require admin action. If you do not see errors when running `CREATE TABLE ... USING JDBC ...`, the feature is likely enabled.

If you are unsure, contact your Databricks workspace admin to confirm that the JDBC table feature is enabled for Unity Catalog in your workspace.